In [1]:
import pandas as pd
import numpy as np
import os,sys
os.chdir('C:/Users/arpit.goel/Documents/Projects/Kaggle/25_KingsOfML')
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# df_train=pd.read_csv('01.RawData/train.csv')
# df_test=pd.read_csv('01.RawData/test_BDIfz5B.csv')
# df_cmpgn=pd.read_csv('01.RawData/campaign_data.csv')
# df_sample=pd.read_csv('01.RawData/sample_submission_4fcZwvQ.csv')
# df_train['send_date']=pd.to_datetime(df_train['send_date'],format='%d-%m-%Y %H:%M')
# df_test['send_date']=pd.to_datetime(df_test['send_date'],format='%d-%m-%Y %H:%M')
# df_train.sort_values(by=['user_id','send_date'],inplace=True)
# df_test.sort_values(by=['user_id','send_date'],inplace=True)
# store=pd.HDFStore('01.RawData/DataStore.h5')
# store['train']=df_train
# store['test']=df_test
# store['cmpgn']=df_cmpgn
# store['sample']=df_sample
# store.close()

df_train=pd.read_hdf('01.RawData/DataStore.h5', 'train')
df_test=pd.read_hdf('01.RawData/DataStore.h5', 'test')
df_cmpgn=pd.read_hdf('01.RawData/DataStore.h5', 'cmpgn')
df_sample=pd.read_hdf('01.RawData/DataStore.h5', 'sample')

In [142]:
a='07-08-2015 sadd'
a[11:13]

'sa'

In [133]:
df_train.head()

,id,user_id,campaign_id,send_date,is_open,is_click
621721,32_5,5,32,2017-07-19 12:02:00,0,0
38232,53_5,5,53,2017-11-06 22:33:00,0,0
629867,30_6,6,30,2017-07-05 14:07:00,1,0
196175,32_6,6,32,2017-07-19 12:13:00,1,0
439258,34_6,6,34,2017-08-01 22:18:00,0,0


In [134]:
df_test.head()

,id,campaign_id,user_id,send_date
762628,55_5,55,5,2018-01-09 15:35:00
40347,60_5,60,5,2018-01-11 16:04:00
624874,66_5,66,5,2018-02-15 15:51:00
326091,70_5,70,5,2018-02-16 15:32:00
285170,73_5,73,5,2018-02-22 16:47:00


In [26]:
#Get campaign type and campaign rank
comm_type=df_cmpgn.set_index('campaign_id')['communication_type']
comm_type=comm_type.map(lambda x: 'Upcoming Events' if x in ['Others','Webinar'] else x)
comm_type=comm_type.astype('category').cat.codes

campaign_dates=pd.concat([df_train[['campaign_id','send_date']],df_test[['campaign_id','send_date']]])
campaign_dates=campaign_dates.groupby(['campaign_id'])['send_date'].min().to_frame('first_send_date').reset_index()
campaign_dates['campaign_type']=campaign_dates['campaign_id'].map(comm_type)
campaign_dates['campaign_rank1']=campaign_dates['first_send_date'].rank()
campaign_dates['campaign_rank2']=campaign_dates.groupby(['campaign_type'])['campaign_rank1'].rank()
campaign_dates=campaign_dates[['campaign_id','campaign_type','campaign_rank1','campaign_rank2']]
campaign_dates.index=campaign_dates.campaign_id

In [25]:
#Training Data Signals
signal_raw=df_train.copy()
signal_raw['rcv']=1
signal_raw=pd.merge(signal_raw,campaign_dates,on='campaign_id')

#User Signals
signals_1=signal_raw.set_index(['user_id','campaign_rank1'])[['rcv','is_open','is_click']]
signals_1=signals_1.stack().unstack(1).fillna(0).cumsum(axis=1)
cols1=signals_1.columns
signals_1=signals_1[cols1[:-1]]
signals_1.columns=cols1[1:]
signals_1=signals_1.stack().unstack(1).fillna(0)
signals_1.columns=['sig_rcv','sig_open','sig_click']

#User-Campaign Signals
signals_2=signal_raw.set_index(['user_id','campaign_type','campaign_rank2'])[['rcv','is_open','is_click']]
signals_2=signals_2.stack().unstack(2).fillna(0).cumsum(axis=1)
cols1=signals_2.columns
signals_2=signals_2[cols1[:-1]]
signals_2.columns=cols1[1:]
signals_2=signals_2.stack().unstack(2).fillna(0)
signals_2.columns=['sig_rcv_type','sig_open_type','sig_click_type']

signal_master=signal_raw[['id','user_id','campaign_id','campaign_rank1','campaign_rank2','is_click','campaign_type']]
signal_master=pd.merge(signal_master,signals_1.reset_index(),on=['user_id','campaign_rank1'],how='left')
signal_master=pd.merge(signal_master,signals_2.reset_index(),on=['user_id','campaign_type','campaign_rank2'],how='left')
signal_master=signal_master.fillna(0).set_index(['user_id','campaign_id','id'])
signal_master=pd.concat([signal_master,pd.get_dummies(signal_master['campaign_type'],prefix='sig_cmpgn_type')],axis=1)
signal_train=signal_master.drop(labels=['campaign_rank1','campaign_rank2','campaign_type'],axis=1)

In [55]:
#Test Data Signals

#User Signals
signals_1=signal_raw.groupby(['user_id'])[['rcv','is_open','is_click']].sum()
signals_1.columns=['sig_rcv','sig_open','sig_click']

#User-Campaign Signals
signals_2=signal_raw.groupby(['user_id','campaign_type'])[['rcv','is_open','is_click']].sum()
signals_2.columns=['sig_rcv_type','sig_open_type','sig_click_type']

signal_master=df_test[['id','user_id','campaign_id']]
signal_master['campaign_type']=signal_master['campaign_id'].map(campaign_dates['campaign_type'])
signal_master=pd.merge(signal_master,signals_1.reset_index(),on=['user_id'],how='left')
signal_master=pd.merge(signal_master,signals_2.reset_index(),on=['user_id','campaign_type'],how='left')
signal_master=signal_master.fillna(0).set_index(['user_id','campaign_id','id'])
signal_master=pd.concat([signal_master,pd.get_dummies(signal_master['campaign_type'],prefix='sig_cmpgn_type')],axis=1)
signal_test=signal_master.drop(labels=['campaign_type'],axis=1)


In [56]:
#Master Signal Set
signal_master=pd.concat([signal_train,signal_test]).fillna(0)

In [57]:
#Add Historic Open Rate, Click Rate and clicks per open
for sigset in ['','_type']:
    signal_master['sig_click_rate%s'%sigset]=1.0*signal_master['sig_click%s'%sigset]/(signal_master['sig_rcv%s'%sigset]+0.0001)
    signal_master['sig_open_rate%s'%sigset]=1.0*signal_master['sig_open%s'%sigset]/(signal_master['sig_rcv%s'%sigset]+0.0001)
    signal_master['sig_click_per_open%s'%sigset]=1.0*signal_master['sig_click%s'%sigset]/(signal_master['sig_open%s'%sigset]+0.0001)

In [88]:
#Add campaign signals from campaign contents
import re,string
def get_free_text_vars(df,var,label):
    df['sig_ft_0_%s'%label]=df[var].astype(np.str).str.len()                                                             # Num characters
    df['sig_ft_2_%s'%label]=df[var].astype(np.str).map(lambda x: re.sub('[^#$%&*:;<=>?@\^_`|~]','',x)).str.len()         # Num Special Characters
    df['sig_ft_3_%s'%label]=df[var].astype(np.str).map(lambda x: re.sub('[^0-9]','',x)).str.len()                        # Num numerals
    df['sig_ft_4_%s'%label]=df[var].astype(np.str).map(lambda x: re.sub('[^a-z]','',x)).str.len()                        # Num lower case
    df['sig_ft_5_%s'%label]=df[var].astype(np.str).map(lambda x: re.sub('[^A-Z]','',x)).str.len()                        # Num upper case
    df['sig_ft_6_%s'%label]=df[var].astype(np.str).map(lambda x: re.sub('[^ ]','',x)).str.len()                          # Num spaces
    return df

cmpgn_sigs=df_cmpgn.set_index(['campaign_id'])
cmpgn_sigs=get_free_text_vars(cmpgn_sigs,'subject','sub')
cmpgn_sigs=get_free_text_vars(cmpgn_sigs,'email_body','email')
cmpgn_sigs.rename(columns={'total_links':'sig_links1','no_of_internal_links':'sig_links2',
        'no_of_images':'sig_images','no_of_sections':'sig_sections'},inplace=True)
cmpgn_sigs1=cmpgn_sigs[[x for x in cmpgn_sigs.columns if 'sig' in x]]

#Add campaign level signals from campaign population
cmpgn_sigs=signal_master[['sig_rcv','sig_open','sig_click']].reset_index()
cmpgn_sigs['rcv']=1
cmpgn_sigs['sig_cmpgn_rcv_0']=1
cmpgn_sigs['sig_cmpgn_rcv_1']=(cmpgn_sigs['sig_rcv']>=1).astype(np.int64)
cmpgn_sigs['sig_cmpgn_rcv_2']=(cmpgn_sigs['sig_rcv']>=2).astype(np.int64)
cmpgn_sigs['sig_cmpgn_open_1']=(cmpgn_sigs['sig_open']>=1).astype(np.int64)
cmpgn_sigs['sig_cmpgn_open_2']=(cmpgn_sigs['sig_open']>=2).astype(np.int64)
cmpgn_sigs['sig_cmpgn_click_1']=(cmpgn_sigs['sig_click']>=1).astype(np.int64)
cmpgn_sigs['sig_cmpgn_click_2']=(cmpgn_sigs['sig_click']>=2).astype(np.int64)

vars1=['sig_cmpgn_%s_%d'%(x,y) for x in ['rcv','open','click'] for y in [1,2]]
cmpgn_sigs=cmpgn_sigs.groupby('campaign_id')[['rcv','sig_cmpgn_rcv_0']+vars1].sum()
for var in vars1:
    cmpgn_sigs[var.replace('cmpgn_','cmpgn_rate_')]=1.0*cmpgn_sigs[var]/cmpgn_sigs['rcv']
cmpgn_sigs2=cmpgn_sigs[[x for x in cmpgn_sigs.columns if 'sig' in x]]
cmpgn_sigs=pd.concat([cmpgn_sigs1,cmpgn_sigs2],axis=1)


In [129]:
#Add campaign signals to master
# signal_master1=pd.merge(signal_master.reset_index(),cmpgn_sigs2,left_on=['campaign_id'],right_index=True)
# signal_master1.set_index(['user_id','campaign_id','id'],inplace=True)
signal_master1=signal_master

In [130]:
#Get datasets for modelling. Turn the below parameter to False while subitting.
validation=True

train_campaigns=range(29,47) if validation==True else range(29,55)
valid_campaigns=range(47,55) if validation==True else range(55,81)

train=signal_master1[signal_master1.index.get_level_values(1).isin(train_campaigns)]
valid=signal_master1[signal_master1.index.get_level_values(1).isin(valid_campaigns)]

signals=[x for x in signal_master1.columns if 'sig' in x]
train_X=train[signals]
train_Y=train['is_click']
valid_X=valid[signals]
valid_Y=valid['is_click']
print [x.shape for x in [train_X,train_Y,valid_X,valid_Y]]

[(571021, 17), (571021L,), (452170, 17), (452170L,)]


In [131]:
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

lgb_train = lgb.Dataset(train_X, train_Y)
if validation==True:
    lgb_test = lgb.Dataset(valid_X, valid_Y)
    watchlist = [lgb_train,lgb_test]
else:
    watchlist = [lgb_train]
    
for param in [1]:
    params = {
        'learning_rate': 0.001,
        'max_depth':5,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'num_leaves': 2**4-1,
        'verbose': -1,
        'data_random_seed': 1,
        'bagging_fraction': 0.85, 
        'bagging_freq': 2,
        'feature_fraction': 0.6,
        'min_data_in_leaf': 100,
    }
    
    gbm = lgb.train(params,lgb_train,num_boost_round=50,valid_sets=watchlist,verbose_eval=10)
    print (param,roc_auc_score(train_Y, gbm.predict(train_X)))
    print (param,roc_auc_score(valid_Y, gbm.predict(valid_X)))
    


[10]	training's auc: 0.69308	valid_1's auc: 0.642189
[20]	training's auc: 0.691394	valid_1's auc: 0.641919
[30]	training's auc: 0.690767	valid_1's auc: 0.643682
[40]	training's auc: 0.694249	valid_1's auc: 0.645304
[50]	training's auc: 0.694325	valid_1's auc: 0.646931
(1, 0.69432547260203192)
(1, 0.64693105572388276)


In [121]:
valid_X['is_click']=gbm.predict(valid_X)
valid_X.reset_index()[['id','is_click']].to_csv('03.Submission/05.LGBMCampaign.csv',index=False)


In [127]:
#Add campaign level signals from campaign population
cmpgn_sigs=master[['sum_rcv','sum_is_open','sum_is_click','campaign_id']]
cmpgn_sigs['rcv']=1
cmpgn_sigs['sig_cmpgn_rcv_0']=1
cmpgn_sigs['sig_cmpgn_rcv_1']=(cmpgn_sigs['sum_rcv']>=1).astype(np.int64)
cmpgn_sigs['sig_cmpgn_rcv_2']=(cmpgn_sigs['sum_rcv']>=2).astype(np.int64)
cmpgn_sigs['sig_cmpgn_open_1']=(cmpgn_sigs['sum_is_open']>=1).astype(np.int64)
cmpgn_sigs['sig_cmpgn_open_2']=(cmpgn_sigs['sum_is_open']>=2).astype(np.int64)
cmpgn_sigs['sig_cmpgn_click_1']=(cmpgn_sigs['sum_is_click']>=1).astype(np.int64)
cmpgn_sigs['sig_cmpgn_click_2']=(cmpgn_sigs['sum_is_click']>=2).astype(np.int64)

vars1=['sig_cmpgn_%s_%d'%(x,y) for x in ['rcv','open','click'] for y in [1,2]]
cmpgn_sigs=cmpgn_sigs.groupby('campaign_id')[['rcv','sig_cmpgn_rcv_0']+vars1].sum()
for var in vars1:
    cmpgn_sigs[var.replace('cmpgn_','cmpgn_rate_')]=1.0*cmpgn_sigs[var]/cmpgn_sigs['rcv']
cmpgn_sigs2=cmpgn_sigs[[x for x in cmpgn_sigs.columns if 'sig' in x]]
cmpgn_sigs=pd.concat([cmpgn_sigs1,cmpgn_sigs2],axis=1)

#Add campaign signals to master
master=pd.merge(master,cmpgn_sigs2,left_on=['campaign_id'],right_index=True)
    
def get_train_test_data(valid=True):
    signals=[x for x in master.columns if x[:3] in ['sum','com','sig'] or x in ['campaign_rank']]
    train_months=[8,9] if valid==True else [8,9,10,11,12]
    valid_months=[10,11,12] if valid==True else [13]
    train_X=master[master.month.isin(train_months)][signals]
    valid_X=master[master.month.isin(valid_months)][signals]
    train_Y=master[master.month.isin(train_months)][['is_click','is_open']]
    valid_Y=master[master.month.isin(valid_months)][['is_click','is_open']]
    return train_X,valid_X,train_Y,valid_Y
        
master.set_index(['id','user_id','campaign_id'],inplace=True)
train_X,valid_X,train_Y,valid_Y=get_train_test_data(valid=False)

from sklearn.metrics import roc_auc_score
import lightgbm as lgb

params = {
        'learning_rate': 0.001,
        'max_depth':4,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'num_leaves': 2**3-1,
        'verbose': -1,
        'data_random_seed': 1,
        'bagging_fraction': 0.7,
        'bagging_freq': 5,
        'feature_fraction': 0.6,
        'min_data_in_leaf': 250,
    }

lgb_train = lgb.Dataset(train_X, train_Y['is_click'])
watchlist = [lgb_train]
gbm = lgb.train(params,lgb_train,num_boost_round=25,valid_sets=watchlist,verbose_eval=5)

valid_X['is_click']=gbm.predict(valid_X)
valid_X.reset_index()[['id','is_click']].to_csv('03.Submission/05.LGBMCampaign.csv',index=False)


C:\Users\arpit.goel\AppData\Local\Continuum2\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\arpit.goel\AppData\Local\Continuum2\Anaconda2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\arpit.goel\AppData\Local\Continuum2\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying

[5]	training's auc: 0.66944
[10]	training's auc: 0.670283
[15]	training's auc: 0.670115
[20]	training's auc: 0.674219
[25]	training's auc: 0.673931


In [128]:
valid_X['is_click']=gbm.predict(valid_X)
valid_X.reset_index()[['id','is_click']].to_csv('03.Submission/05.LGBMCampaign.csv',index=False)


In [166]:
%run 00.Scripts/02.LGBMBase.py

00.Scripts/02.LGBMBase.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  cmpgn_sigs['rcv']=1
00.Scripts/02.LGBMBase.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  cmpgn_sigs['sig_cmpgn_rcv_0']=1
00.Scripts/02.LGBMBase.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  cmpgn_sigs['sig_cmpgn_rcv_1']=(cmpgn_si

[5]	training's auc: 0.675448
[10]	training's auc: 0.675422
[15]	training's auc: 0.67572
[20]	training's auc: 0.676343
[25]	training's auc: 0.67631


In [196]:
train_X,valid_X,train_Y,valid_Y=get_train_test_data(valid=True)


In [168]:
train_X.columns

Index([u'communication_type', u'sum_rcv', u'sum_is_click', u'sum_is_open',
       u'sum_rcv_0', u'sum_is_click_0', u'sum_is_open_0', u'sum_rcv_3',
       u'sum_is_click_3', u'sum_is_open_3', u'sum_rcv_5', u'sum_is_click_5',
       u'sum_is_open_5', u'sum_rcv_2', u'sum_is_click_2', u'sum_is_open_2',
       u'sum_rcv_6', u'sum_is_click_6', u'sum_is_open_6', u'sum_rcv_4',
       u'sum_is_click_4', u'sum_is_open_4', u'sum_rcv_1', u'sum_is_click_1',
       u'sum_is_open_1', u'campaign_rank', u'sum_open_rate', u'sum_click_rate',
       u'sum_click_per_open', u'sum_open_rate_0', u'sum_click_rate_0',
       u'sum_click_per_open_0', u'sum_open_rate_3', u'sum_click_rate_3',
       u'sum_click_per_open_3', u'sum_open_rate_5', u'sum_click_rate_5',
       u'sum_click_per_open_5', u'sum_open_rate_2', u'sum_click_rate_2',
       u'sum_click_per_open_2', u'sum_open_rate_6', u'sum_click_rate_6',
       u'sum_click_per_open_6', u'sum_open_rate_4', u'sum_click_rate_4',
       u'sum_click_per_open_4', u's

In [197]:
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

idv=['communication_type', u'sum_rcv', u'sum_is_click', u'sum_is_open',u'sum_open_rate', u'sum_click_rate',
     u'sum_click_per_open']
lgb_train = lgb.Dataset(train_X[idv], train_Y['is_click'])
lgb_test = lgb.Dataset(valid_X[idv], valid_Y['is_click'])
watchlist = [lgb_train,lgb_test]

for param in [3]:
    params = {
        'learning_rate': 0.04,
        'max_depth':param,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'num_leaves': 2**3-1,
        'verbose': -1,
        'data_random_seed': 1,
        'bagging_fraction': 0.7,
        'bagging_freq': 5,
        'feature_fraction': 0.6,
        'min_data_in_leaf': 250,
    }
    
    gbm = lgb.train(params,lgb_train,num_boost_round=50,valid_sets=watchlist,verbose_eval=10)
    print (param,roc_auc_score(train_Y['is_click'], gbm.predict(train_X[idv])))
    print (param,roc_auc_score(valid_Y['is_click'], gbm.predict(valid_X[idv])))

[10]	training's auc: 0.673649	valid_1's auc: 0.65377
[20]	training's auc: 0.683417	valid_1's auc: 0.650811
[30]	training's auc: 0.684609	valid_1's auc: 0.651076
[40]	training's auc: 0.684584	valid_1's auc: 0.650841
[50]	training's auc: 0.684436	valid_1's auc: 0.650986
(3, 0.68443648351094821)
(3, 0.65098555213929621)


In [199]:
valid_Y['tgt']=gbm.predict(valid_X[idv])
for name,group in valid_Y.reset_index().groupby('campaign_id'):
    print name,roc_auc_score(group['is_click'], group['tgt'])

48 0.659423945885
50 0.659767440121
51 0.68226895284
52 0.664032161966
53 0.583005505273
54 0.661530371877


In [188]:
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

idv=['sum_click_per_open', u'sum_rcv', u'sum_open_rate', u'sum_is_click',u'sig_cmpgn_rate_open_1',u'sig_cmpgn_rate_rcv_1'
     ]
#
# idv=train_X.columns
lgb_train = lgb.Dataset(train_X[idv], train_Y['is_click'])
lgb_test = lgb.Dataset(valid_X[idv], valid_Y['is_click'])
watchlist = [lgb_train,lgb_test]

for param in [3]:
    params = {
        'learning_rate': 0.04,
        'max_depth':param,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'num_leaves': 2**3-1,
        'verbose': -1,
        'data_random_seed': 1,
        'bagging_fraction': 0.7,
        'bagging_freq': 5,
        'feature_fraction': 0.6,
        'min_data_in_leaf': 250,
    }
    
    gbm = lgb.train(params,lgb_train,num_boost_round=5,valid_sets=watchlist,verbose_eval=1)
    print (param,roc_auc_score(train_Y['is_click'], gbm.predict(train_X[idv])))
    print (param,roc_auc_score(valid_Y['is_click'], gbm.predict(valid_X[idv])))

[1]	training's auc: 0.631314	valid_1's auc: 0.624765
[2]	training's auc: 0.678066	valid_1's auc: 0.641265
[3]	training's auc: 0.711019	valid_1's auc: 0.643449
[4]	training's auc: 0.703466	valid_1's auc: 0.645585
[5]	training's auc: 0.708287	valid_1's auc: 0.652207
(3, 0.7082872185982183)
(3, 0.65220742862369541)


In [189]:
pd.Series(gbm.feature_importance(),index=idv).sort_values(ascending=False)

sum_open_rate            7
sum_is_click             6
sum_click_per_open       5
sig_cmpgn_rate_rcv_1     4
sig_cmpgn_rate_open_1    4
sum_rcv                  4
dtype: int32

In [190]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression()
clf.fit(train_X[idv], train_Y['is_click'])
print (roc_auc_score(train_Y['is_click'], clf.predict(train_X[idv])))
print (roc_auc_score(valid_Y['is_click'], clf.predict(valid_X[idv])))

0.500499652597
0.505578653249


In [195]:
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

params = {
        'learning_rate': 0.04,
        'max_depth':3,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'num_leaves': 2**3-1,
        'verbose': -1,
        'data_random_seed': 1,
        'bagging_fraction': 0.7,
        'bagging_freq': 5,
        'feature_fraction': 0.6,
        'min_data_in_leaf': 250,
    }

idv=['communication_type', u'sum_rcv', u'sum_is_click', u'sum_is_open',u'sum_open_rate', u'sum_click_rate',
     u'sum_click_per_open']
#
# idv=train_X.columns
lgb_train = lgb.Dataset(train_X[idv], train_Y['is_click'])
watchlist = [lgb_train]
gbm = lgb.train(params,lgb_train,num_boost_round=50,valid_sets=watchlist,verbose_eval=10)

valid_X['is_click']=gbm.predict(valid_X[idv])
valid_X.reset_index()[['id','is_click']].to_csv('03.Submission/07.LGBMTest1.csv',index=False)


[10]	training's auc: 0.659333
[20]	training's auc: 0.66588
[30]	training's auc: 0.666205
[40]	training's auc: 0.666083
[50]	training's auc: 0.666263


In [214]:
t1=df_train[df_train['campaign_id'].isin([49,41])]
t1=t1.set_index(['user_id','campaign_id'])[['rcv','is_open','is_click']].unstack().fillna(0)
t1.columns=t1.columns.get_level_values(0)+'_'+t1.columns.get_level_values(1).astype(str)
t1.reset_index().groupby(t1.columns.tolist())['user_id'].count().to_frame('count').to_clipboard()


In [234]:
t1=df_train.copy()
t1['key']=t1.user_id.map(lambda x: '%07d'%(x))
t1['key']=t1['key'].str[5]
t1['key']=t1['send_date'].dt.minute
# t1['key']=t1['key'].str[5]
t1.groupby(['key','campaign_id']).apply(lambda x: pd.Series({'rcv':x['rcv'].sum(),'click_rate':x['is_click'].mean()})).to_clipboard()

In [235]:
%run 00.Scripts/05.LGBMSelectedVars.py

KeyError: 'is_click'

In [237]:
train_X,valid_X,train_Y,valid_Y=get_train_test_data(valid=True)
random=np.random.rand(len(train_X))
train_X=train_X[()&()]

In [247]:
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

idv=['sum_click_per_open', u'sum_rcv', u'sum_open_rate', u'sum_is_click',u'sig_cmpgn_rate_open_1',u'sig_cmpgn_rate_rcv_1'
     ]

idv=train_X.columns
lgb_train = lgb.Dataset(train_X[idv], train_Y)
lgb_test = lgb.Dataset(valid_X[idv], valid_Y)
watchlist = [lgb_train,lgb_test]

for param in [1]:
    params = {
        'learning_rate': 0.01,
        'max_depth':3,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'num_leaves': 2**3-1,
        'verbose': -1,
        'data_random_seed': 1,
        'bagging_fraction': 0.7,
        'bagging_freq': 2,
        'feature_fraction': 0.6,
        'min_data_in_leaf': 250,
    }
    
    gbm = lgb.train(params,lgb_train,num_boost_round=10,valid_sets=watchlist,verbose_eval=1)
    print (param,roc_auc_score(train_Y, gbm.predict(train_X[idv])))
    print (param,roc_auc_score(valid_Y, gbm.predict(valid_X[idv])))

[1]	training's auc: 0.655015	valid_1's auc: 0.640065
[2]	training's auc: 0.658845	valid_1's auc: 0.656193
[3]	training's auc: 0.658259	valid_1's auc: 0.653704
[4]	training's auc: 0.659941	valid_1's auc: 0.655541
[5]	training's auc: 0.659938	valid_1's auc: 0.655195
[6]	training's auc: 0.664826	valid_1's auc: 0.654539
[7]	training's auc: 0.665088	valid_1's auc: 0.654939
[8]	training's auc: 0.664697	valid_1's auc: 0.654787
[9]	training's auc: 0.666583	valid_1's auc: 0.656649
[10]	training's auc: 0.667564	valid_1's auc: 0.654782
(1, 0.66756442094074331)
(1, 0.65478172737956553)


In [248]:
print pd.Series(gbm.feature_importance(),index=idv).sort_values(ascending=False)

communication_type     11
ever_click_per_open    10
ever_open               8
ever_open_rate          7
ever_rcv                6
ever_click_rate         4
last_open               4
last_rcv                4
last_open_rate          2
ever_click              2
last_click_rate         1
last_click              1
last_click_per_open     0
dtype: int32
